In [ ]:
import utils
import embeddings
from experiments import Experiments
import numpy as np

In [ ]:
path_to_dataset = 'fakerecogna_abstrativo.xlsx'

# formatting the dataset
utils.FakeRecogna2(path_to_dataset)

dataset = utils.load()

In [ ]:
# generate embeddings
doc2vec = embeddings.Doc2Vec()
word2vec = embeddings.Word2Vec()

vectors = [np.zeros((dataset.shape[0],1024)),
           np.zeros((dataset.shape[0],100))]

vectors[0] = doc2vec.getVector(dataset)
np.save('.\\fakerecogna2\\vectors\\BERT.npy', vectors[0])
vectors[1] = word2vec.getVector(dataset)
np.save('.\\fakerecogna2\\vectors\\word2vec.npy', vectors[1])

In [ ]:
# loading the dataset
models = ['BERT','word2vec']
dataset, vectors = utils.load(models)

In [ ]:
# preparing the experiments
dataset = dataset.loc[(dataset.date >= '2020-01-01') & (dataset.date <= '2021-12-31')]
exp = Experiments(dataset, size=10,N=1)

In [ ]:
# tests on chronological data
results_BERT = exp.different_distribution(vectors['BERT'])
results_word2vec = exp.different_distribution(vectors['word2vec'])

In [ ]:
# tests on random data
results_BERT = exp.same_distribution_test(vectors['BERT'])
results_word2vec = exp.same_distribution_test(vectors['word2vec'])

In [ ]:
import pandas as pd

# WMD on chronological data
WMD = exp.WMD()
dt_wmd = pd.DataFrame(columns=['label', 'week', 'mwmd'])
weeks = pd.date_range('2020-01-01','2021-12-31', freq='2W').to_pydatetime()

for i in range(1,WMD.shape[1]):
    temp = WMD[0][i].mean(axis=1).mean(axis=2)[i-1,:].T
    dt_wmd = pd.concat([dt_wmd,pd.DataFrame({'mwmd':temp, 'label':True, 'week':weeks[i-1]})])
for i in range(1,WMD.shape[1]):
    temp = WMD[1][i].mean(axis=1).mean(axis=2)[i-1,:].T
    dt_wmd = pd.concat([dt_wmd,pd.DataFrame({'mwmd':temp, 'label':False, 'week':weeks[i-1]})])
dt_wmd.reset_index(drop=True, inplace=True)

In [ ]:
# WMD on random data
WMD = exp.same_distribution_WMD()
dt_wmd = pd.DataFrame(columns=['label', 'week', 'mwmd'])
weeks = pd.date_range('2020-01-01','2021-12-31', freq='2W').to_pydatetime()

for i in range(1,WMD.shape[1]):
    temp = WMD[0][i].mean(axis=1).mean(axis=2)[i-1,:].T
    dt_wmd = pd.concat([dt_wmd,pd.DataFrame({'mwmd':temp, 'label':True, 'week':weeks[i-1]})])
for i in range(1,WMD.shape[1]):
    temp = WMD[1][i].mean(axis=1).mean(axis=2)[i-1,:].T
    dt_wmd = pd.concat([dt_wmd,pd.DataFrame({'mwmd':temp, 'label':False, 'week':weeks[i-1]})])
dt_wmd.reset_index(drop=True, inplace=True)